In [1]:
# run this first
# This will help you quickly gather base demand information from your system

from epyt import epanet
import pandas as pd
import numpy as np

# read in the relevant .inp file (assuming you have already created your pressure zones)
d = epanet('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/Gill_Spring2024/WhitesburgHydraulicFiles/EPANET_Files/Whitesburg_GillUpdates.inp')
patternIndex = d.getNodeDemandPatternIndex()
baseDemand = d.getNodeBaseDemands()
patternIndex = patternIndex[1][:]
baseDemand = baseDemand[1][:]

df = pd.DataFrame([patternIndex, baseDemand])

#This has a place holder, ignore the first number reported below. Additionally, please remember that this is indexed.
# if the first pattern in your .inp file is named '2' you'll have to be mindful.
count = [0,0,0,0,0,0,0]

for i in range(6):
    for j in range(308):
        if df.iloc[0][j] == i:
            add = df.iloc[1][j]
            count[i] = count[i] + add
count

#these results will inform me as to how to set up my mass balance

#for the tanks that we are not maipulating the pattern for
patterns = d.getPattern()
demandZone4 = patterns[3] * count[4]
demandZone5 = patterns[4] * count[5]
demandZone6 = patterns[5] * count[6]


count[2]

EPANET version 20200 loaded (EPyT version 1.0.7).
Input File Whitesburg_GillUpdates.inp loaded successfully.



47.035999999999994

In [2]:
#The number of inputs here will have to change depending on the number of demand patterns you are calibrating

def massFlowRate(demandPattern1, demandPattern2, demandPattern3):
    

    import numpy as np

    #This is just so that we can have an initial flow rate coming from the treatment plant for our mass balance
    
    d.setPattern(2, demandPattern1)
    d.setPattern(3, demandPattern2)
    d.setPattern(1, demandPattern3)
    
    d.openHydraulicAnalysis()
    d.initializeHydraulicAnalysis()
    Series = d.getComputedHydraulicTimeSeries()
    d.closeHydraulicAnalysis()

    #capturing the proper head
    Head = Series.Head[:,[306,307,304]]
    #capturing the proper flows (from the WTP)
    Flow = Series.Flow[:,341]
    #capturing the time
    timeCheck = Series.Time[:]/3600


    resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], Head[:,2]], columns = timeCheck )

    resultDataFrame

    #okay so flow needs to be processed a little more for Whitesburg. The reason for this is because if we turn a pump on at 1:15 the flow rate at 1:00
    # is 0. Thats an issue because for 45 minutes we have flow coming into the system that we are not accounting for. Therefore, we
    # need to find the average flow over the hour to plug into the mass balance equation. please follow logic below:

    #Here we are computing the average flow for each hour
    averageFlowRate = [0] * 24
    for h in range(23):
        hourlyTotalVolume = 0
        hourlyAverage = 0
        for idx, t in enumerate(timeCheck):
            if h <= t < (h + 1):
                timeDifference = (timeCheck[idx+1] - timeCheck[idx]) *60
                flowAtTime = Flow[idx]
                flowVolume = timeDifference * flowAtTime
                hourlyTotalVolume = hourlyTotalVolume + flowVolume
                hourlyAverage = hourlyTotalVolume/60
                #print(f"index {idx} belongs to hour {h}, with a difference of {timeDifference} minutes a running total flow volume of {flowVolume} and a running average of {hourlyAverage}")
        averageFlowRate[h] = hourlyAverage
        hourlyTotalVolume = 0
    return averageFlowRate  

In [4]:
def modelResults():
    import numpy as np

    #This is just so that we can have an initial flow rate coming from the treatment plant for our mass balance

    d.openHydraulicAnalysis()
    d.initializeHydraulicAnalysis()
    Series = d.getComputedHydraulicTimeSeries()
    d.closeHydraulicAnalysis()

    #capturing the proper head
    Head = Series.Head[:,[306,307,304]]
    #capturing the proper flows (from the WTP)
    Flow = Series.Flow[:,341]
    #capturing the time
    timeCheck = Series.Time[:]/3600


    resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], Head[:,2]], columns = timeCheck )

    return resultDataFrame
resultDataFrame = modelResults()

In [5]:
# Testing generic Box - Complex on Whitesburg Kentucky Data

from epyt import epanet
import numpy as np
import pandas as pd
import math

# Find the file path of the EPANET file in your directory
d = epanet('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/Gill_Spring2024/WhitesburgHydraulicFiles/EPANET_Files/Whitesburg_GillUpdates.inp')

# Read in generic tank data, change the path to match your system
dfTanks = pd.read_excel('C:/Users/apgi227/OneDrive - University of Kentucky/Documents/GitHub/FinalProject_CE610/exampleTankLevels.xlsx')

# This is where we are storing the solutions for the demand factors
resultPattern1 = [1] * 24
resultPattern2 = [1] * 24
resultPattern3 = [1] * 24

# the total length of the EPS
for h in range(23):
    
    # create random positive demand factors (here we will create n-1 factors where n is the number of zones in the system)
    demandFactors = [[0.25, 1], [1, 0.25], [0.25, 0.25]]
    
    # change depending on how many tanks are in sim (this is for the tank levels at the end of the hour we are analyzing)
    sawmillRoad = dfTanks.iloc[h+1,0]
    tunnelHill = dfTanks.iloc[h+1,1]
    colley = dfTanks.iloc[h+1,2]
    
    #Here we are putting a data frame together for our change in tanks levels for processing the mass balance
    
    flowSawmill = (((math.pi/4) * ( 50**2) * (resultDataFrame.iloc[1][h] - sawmillRoad)) * 7.48)/60
    flowTunnelHill = (((math.pi/4) * (25**2) * (resultDataFrame.iloc[2][h] - tunnelHill)) * 7.48)/60
    flowColley = (((math.pi/4) * (50**2) * (resultDataFrame.iloc[3][h] - colley)) * 7.48)/60
    totalTankFlow = flowSawmill + flowTunnelHill + flowColley
    
    # Whitesburg is weird, if I did (n-1) points then I would only have 2 of them so I am just creating a third point 
    # so that the box - complex works. That won't create any issues - we are still working in 2-D space (pattern 1 and 2) and 
    # creating a third pattern based off the mass balance
    
    pointsInSimplex = [1, 2, 3]
    
    wtpFlow = massFlowRate(resultPattern1, resultPattern2, resultPattern3)
    averageFlowRate = wtpFlow[h]
    
    #change depending on the number of points in the simplex
    for i in range(3):

            # depending on the number of zones, create demand patterns
            resultPattern1[h] = demandFactors[i][0]
            resultPattern2[h] = demandFactors[i][1]
            resultPattern3[h] = (averageFlowRate + (totalTankFlow) - (demandZone4[h] + demandZone5[h] + demandZone6[h] + (count[2] * resultPattern1[h]) + (count[3] * resultPattern2[h]) )) / count[1]
            
            # this is complicated but because we don't have actual flow data, changing patterns changes the flow rate out of the
            # plant and therefore we need to account for that variability
            
        # Here we first discover why we are using (n-1) points in the simplex. Because we want to incorporate real data, we are using
        # a mass balance to determine what the nth demand pattern will be. Add or subrtract patterns to ensure that this suits the system.

            # nthPattern[h] = ((volume in at time t + the change in storage of the tanks(flow out is positive)) - (Base Demand Zone 1 * pattern1) - (Base Demand Zone 2 * pattern2) - (Base Demand zone 3 * pattern3)) / Base Demand nth zone

            # Create appropriate number of demand patterns dependant on system. Make sure this indexes properly within the .inp file.
            # for example d.setPattern(1, [pattern1] + dummyPattern) is calling the first indexed demand pattern in the file and not 
            # necessarily the demand pattern with the name of 1.

            d.setPattern(2, resultPattern1)
            d.setPattern(3, resultPattern2)
            d.setPattern(1, resultPattern3)
            
            
            d.openHydraulicAnalysis()
            d.initializeHydraulicAnalysis()
            Series = d.getComputedHydraulicTimeSeries()
            d.closeHydraulicAnalysis()
            
            #capturing the proper head
            Head = Series.Head[:,[306,307,304]]
            #capturing the proper flows (from the WTP)
            Flow = Series.Flow[:,341]
            #capturing the time
            timeCheck = Series.Time[:]/3600

            resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], Head[:,2]], columns = timeCheck )

            # Change the below statements (add or subtract) dpending on how many tanks are in your objective function. 
            # You must also go into the .inp file to figure out the exact index of the tanks in the sim (you can use d.getNodeNameID)
            ModelTank1 = resultDataFrame.iloc[1][h+1]
            ModelTank2 = resultDataFrame.iloc[2][h+1]
            ModelTank3 = resultDataFrame.iloc[3][h+1]
           

            # Update the objective function to match the number of tanks that are in the simulation. 
            Error = ((ModelTank1 - sawmillRoad) ** 2) + ((ModelTank2 - tunnelHill) ** 2) + ((ModelTank3 - colley) ** 2) 
            pointsInSimplex[i] = Error
    
    while (abs((pointsInSimplex[0] - pointsInSimplex[1])) >=.0001) or (abs((pointsInSimplex[0] - pointsInSimplex[2])) >=.0001):
        
        wtpFlow = massFlowRate(resultPattern1, resultPattern2, resultPattern3)
        averageFlowRate = wtpFlow[h]
        
        # change the range to match the number of points in the simplex
         
        # Change depending on the number of points within the simplex (should equal the number of zones)
        if pointsInSimplex[0] <=.0001 or pointsInSimplex[1] <=.0001 or pointsInSimplex[2] <=.0001:
            
            #if we have reached our condition
            minVal, whereMin = min(pointsInSimplex), pointsInSimplex.index(min(pointsInSimplex))

            #store the calibrated demand factors from this hour
            resultPattern1[h] = demandFactors[whereMin][0]
            resultPattern2[h] = demandFactors[whereMin][1]
            resultPattern3[h] = (averageFlowRate + (totalTankFlow) - (demandZone4[h] + demandZone5[h] + demandZone6[h] + (count[2] * resultPattern1[h]) + (count[3] * resultPattern2[h]) )) / count[1] 
            

        else:

            # Change range depending on the number of points in the simplex
            maxVal, whereMax = max(pointsInSimplex), pointsInSimplex.index(max(pointsInSimplex))
            logical = [i for i in range(3) if pointsInSimplex[i] != maxVal]
            
            ph = demandFactors[whereMax]
            
            #add another "demandFactors[logical[0,1,2...]][j]" if number of zones change. Also change the denominator as well as the "in range() statement"
            centroid = [(demandFactors[logical[0]][j] + demandFactors[logical[1]][j]) / 2 for j in range(2)]
            
            
            # change the range to match the number of zones minus 1 (n-1)
            newPoint = [(2.5 * centroid[j]) - (1.5 * ph[j]) for j in range(2)]
            
            #check to see if evaluated nth pattern is greater than 0. Change the "newPoint[]" and add more if the number of zones is increased
            resultPattern3[h] = (averageFlowRate + (totalTankFlow) - (demandZone4[h] + demandZone5[h] + demandZone6[h] + (count[2] * newPoint[0]) + (count[3] * newPoint[1]) )) / count[1]
            
            # Change the size of "newPoint" to match the number of zones minus 1 (n-1)
            while newPoint[0] < 0 or newPoint[1] < 0 or resultPattern3[h] < 0:
                
                # change the range to match the number of zones minus 1 (n-1)
                newPoint = [(0.5 * newPoint[j]) + (0.5 * centroid[j]) for j in range(2)]

                #check to see if evaluated nth pattern is greater than 0. Change the "newPoint[]" and add more if the number of zones is increased
                resultPattern3[h] = (averageFlowRate + (totalTankFlow) - (demandZone4[h] + demandZone5[h] + demandZone6[h] + (count[2] * newPoint[0]) + (count[3] * newPoint[1]) )) / count[1]
            
            # change the number of patterns to match the number of zones. If mass balance remove the comment
            resultPattern1[h] = newPoint[0]
            resultPattern2[h] = newPoint[1]
            resultPattern3[h] = (averageFlowRate + (totalTankFlow) - (demandZone4[h] + demandZone5[h] + demandZone6[h] + (count[2] * newPoint[0]) + (count[3] * newPoint[1]) )) / count[1]

                
            d.setPattern(2, resultPattern1)
            d.setPattern(3, resultPattern2)
            d.setPattern(1, resultPattern3)


            d.openHydraulicAnalysis()
            d.initializeHydraulicAnalysis()
            Series = d.getComputedHydraulicTimeSeries()
            d.closeHydraulicAnalysis()
            
            #capturing the proper head
            Head = Series.Head[:,[306,307,304]]
            #capturing the proper flows (from the WTP)
            Flow = Series.Flow[:,341]
            #capturing the time
            timeCheck = Series.Time[:]/3600

            resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], Head[:,2]], columns = timeCheck )

            #change this to match the tank heads if you change systems
            ModelTank1 = resultDataFrame.iloc[1][h+1]
            ModelTank2 = resultDataFrame.iloc[2][h+1]
            ModelTank3 = resultDataFrame.iloc[3][h+1]

            Error = ((ModelTank1 - sawmillRoad) ** 2) + ((ModelTank2 - tunnelHill) ** 2) + ((ModelTank3 - colley) ** 2)
           
            if Error < maxVal:
                #change the "resultPattern1..resultPatternN" to match the number of zones minus 1 (n-1)
                demandFactors[whereMax] = [resultPattern1[h], resultPattern2[h]]
                pointsInSimplex[whereMax] = Error
            else:

                # change the range to match the number of zones minus 1 (n-1)
                newPoint = [(0.5 * ph[j]) + (0.5 * centroid[j]) for j in range(2)]
                
                # Evaluate the new point
                
                resultPattern1[h] = newPoint[0]
                resultPattern2[h] = newPoint[1]
                resultPattern3[h] = (averageFlowRate + (totalTankFlow) - (demandZone4[h] + demandZone5[h] + demandZone6[h] + (count[2] * newPoint[0]) + (count[3] * newPoint[1]) )) / count[1]

                d.setPattern(2, resultPattern1)
                d.setPattern(3, resultPattern2)
                d.setPattern(1, resultPattern3)
               

                d.openHydraulicAnalysis()
                d.initializeHydraulicAnalysis()
                Series = d.getComputedHydraulicTimeSeries()
                d.closeHydraulicAnalysis()
                
                #capturing the proper head
                Head = Series.Head[:,[306,307,304]]
                #capturing the proper flows (from the WTP)
                Flow = Series.Flow[:,341]
                #capturing the time
                timeCheck = Series.Time[:]/3600

                resultDataFrame = pd.DataFrame([Flow, Head[:,0], Head[:,1], Head[:,2]], columns = timeCheck )

                ModelTank1 = resultDataFrame.iloc[1][h+1]
                ModelTank2 = resultDataFrame.iloc[2][h+1]
                ModelTank3 = resultDataFrame.iloc[3][h+1]

                Error = ((ModelTank1 - sawmillRoad) ** 2) + ((ModelTank2 - tunnelHill) ** 2) + ((ModelTank3 - colley) ** 2)
               
                
                #change the "resultPattern1..resultPatternN" to match the number of zones minus 1 (n-1)
                demandFactors[whereMax] = [resultPattern1[h], resultPattern2[h]]
                pointsInSimplex[whereMax] = Error
            minVal, whereMin = min(pointsInSimplex), pointsInSimplex.index(min(pointsInSimplex))

            #store the calibrated demand factors from this hour
            resultPattern1[h] = demandFactors[whereMin][0]
            resultPattern2[h] = demandFactors[whereMin][1]
            resultPattern3[h] = (averageFlowRate + (totalTankFlow) - (demandZone4[h] + demandZone5[h] + demandZone6[h] + (count[2] * resultPattern1[h]) + (count[3] * resultPattern2[h]) )) / count[1]
    
    print('Demand for hour',(h+1), 'is', resultPattern3[h], 'for colley', resultPattern1[h], 'for tunnelHill and', resultPattern2[h], 'for Sawmill Road and with a total MSE of', min(pointsInSimplex))
    if h ==4:
        print('keyBreak')

EPANET version 20200 loaded (EPyT version 1.0.7).
Input File Whitesburg_GillUpdates.inp loaded successfully.



C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: System may be hydraulically unstable.
  warnings.warn(errmssg.value.decode())


Demand for hour 1 is 0.7733178842595446 for colley 4.762956718342306 for tunnelHill and 1.21072529514149 for Sawmill Road and with a total MSE of 0.08710812358628263


C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(errmssg.value.decode())
C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: Pumps cannot deliver enough flow or head.
  warnings.warn(errmssg.value.decode())


Demand for hour 2 is 1.646586657766011e-05 for colley 8.05200367750939 for tunnelHill and 0.06836391514946442 for Sawmill Road and with a total MSE of 0.12965213658705563


C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: System may be hydraulically unstable.
  warnings.warn(errmssg.value.decode())
C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: System may be hydraulically unstable.
  warnings.warn(errmssg.value.decode())
C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: System may be hydraulically unstable.
  warnings.warn(errmssg.value.decode())
C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: System may be hydraulically unstable.
  warnings.warn(errmssg.value.decode())
C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\epanet.py:11818: UserWarning: WARNING: System may be hydraulically unstable.
  warnings.warn(errmssg.value.decode())
C:\Users\apgi227\AppData\Roaming\Python\Python311\site-packages\epyt\e

KeyboardInterrupt: 

In [ ]:
h

In [26]:
flowTunnelHill

255.79763683385784

In [14]:
wtpFlow

[0.0,
 700.0347061352721,
 894.6059930211403,
 884.8723447567967,
 432.2650853980667,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 901.2338824238856,
 788.716114942718,
 706.8922560292762,
 691.4966521585468,
 421.1327353339804,
 1.386070553209421e-07,
 8.662940957558882e-09,
 1.691980655773219e-11,
 1.057487909858262e-12,
 0]